In [55]:
from functools import lru_cache
from openai import OpenAI
client = OpenAI(api_key=API_KEY)

In [65]:
# Function to call ChatGPT.

@lru_cache(maxsize=None)
def call_gpt(prompt, temperature=0.7):
    response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
      "role": "user",
      "content": prompt
    }
    ],
    temperature=temperature,
    max_tokens=64,
    top_p=1
    )
    return response.choices[0].message.content

In [82]:
import types
import json
from jinja2 import Template

# The implementation of the stack-based language.

class StackLang:
    def __init__(self):
        self.stack = []

    def push(self, value):
        self.stack.append(value)

    def pop(self):
        if self.stack:
            return self.stack.pop()
        raise IndexError("Pop from an empty stack")

    def MAKE_PROMPT(self):
        if len(self.stack) >= 2:
            prompt_template = self.pop()
            params = self.pop()
            template = Template(prompt_template)
            self.push(template.render(params))

        else:
            raise IndexError("CALL-ing a model requires 2 params: prompt and model params.")

    def SWAP(self):
            if len(self.stack) >= 2:
                self.stack[-1], self.stack[-2] = self.stack[-2], self.stack[-1]
            else:
                raise IndexError("Swap needs at least two elements")


    def CALL(self):
        if len(self.stack) >= 2:
            model = self.pop()
            prompt = self.pop()
            print('%% Calling model %s with prompt: %s' %(model, prompt))
            r = call_gpt(prompt, model['temperature'])
            self.push(r)
        else:
            raise IndexError("CALL-ing a model requires 2 params: prompt and model params.")

    def ADD2 (self):
        if len(self.stack) >= 2:
            d1 = self.pop()
            d2 = self.pop()
            if isinstance(d1, dict) and isinstance(d2, dict):
                self.push({**d1, **d2})
            elif isinstance(d1, list) and isinstance(d2, list):
                self.push({d1 + d2})
            elif isinstance(d1, str) and isinstance(d2, list):
                self.push([d1] + d2)
            elif isinstance(d1, list) and isinstance(d2, str):
                self.push(d1 + [d2]) 
            elif isinstance(d1, str) and isinstance(d2, str):
                self.push([d1, d2])
            else:
                raise IndexError("unexpected case")
        else:
            raise IndexError("ADD2 requires 2 params on the stack")


    def JSON(self):
        if len(self.stack) >= 1:
            self.push(json.loads(self.pop()))
        else:
            raise IndexError("JSON requires one param on the stack")
        
    def JSON_ARRAY(self):
        if len(self.stack) >= 1:
            self.push({'array': self.pop()})
        else:
            raise IndexError("JSON_ARRAY requires one param on the stack")
    
                      
    def DUP(self):
        if len(self.stack) >= 1:
            self.push(self.stack[0])
        else:
            raise IndexError("DUP requires one param on the stack")

    def OVER(self):
        if len(self.stack) >= 2:
            self.stack.append(self.stack[-2])
        else:
            raise IndexError("OVER requires 2 params on the stack")
    

    def execute(self, commands):
        #print(self)

        for cmd in commands:
            print(self)
            if cmd.startswith('"""'):
                self.push(cmd)
            elif cmd.startswith('{'):
                self.push(json.loads(cmd))
            else:
                if hasattr(self, cmd):
                    getattr(self, cmd)()
                else:
                    raise ValueError(f"Unknown command: {cmd}")

    def __str__(self):
            return str(self.stack)

In [58]:
# Function to read the file describing the LLM "program". 

def process_file(filename):
    lines = open(filename).readlines()
    clean_lines = []
    i = 0
    while i < len(lines):
        if lines[i].startswith('--'):
            i = i + 1
            continue
        if lines[i].startswith('"""') is False:
            clean_lines.append(lines[i].strip())
            i = i + 1
        else:
            j = i
            while (j < len(lines) and lines[j].strip().endswith('"""') is False):
                j = j + 1
            # clean_lines.append("\n".join(  [k[:-1] for k in lines[i:j+1] ] ) )
            clean_lines.append("\n".join(lines[i:j+1]).strip())
            i = j + 1
    return [k for k in clean_lines if k != '']

In [86]:
# We run the program against one of the files.

lang = StackLang()
commands = process_file('llm3.txt')
lang.execute(commands)

print()

print(lang)


[]
['"""Tell me a joke."""']
['"""Tell me a joke."""', '"""Tell me a joke."""']
['"""Tell me a joke."""', '"""Tell me a joke."""', {'model': 'gpt-3.5', 'temperature': 0.5}]
% Calling model {'model': 'gpt-3.5', 'temperature': 0.5} with prompt: """Tell me a joke."""
['"""Tell me a joke."""', 'Why did the scarecrow win an award? Because he was outstanding in his field!']
['"""Tell me a joke."""', 'Why did the scarecrow win an award? Because he was outstanding in his field!', '"""Tell me a joke."""']
['"""Tell me a joke."""', 'Why did the scarecrow win an award? Because he was outstanding in his field!', '"""Tell me a joke."""', {'model': 'gpt-4', 'temperature': 0.6}]
% Calling model {'model': 'gpt-4', 'temperature': 0.6} with prompt: """Tell me a joke."""
['"""Tell me a joke."""', 'Why did the scarecrow win an award? Because he was outstanding in his field!', "Sure, here's one for you:\n\nWhy couldn't the bicycle stand up by itself?\n\nBecause it was two tired!"]
['"""Tell me a joke."""',